In [666]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import sys
import os
from sklearn.ensemble import RandomForestClassifier


# Trova il percorso assoluto della cartella "esotic" (dove si trova il tuo ipynb)
current_dir = os.path.dirname(os.path.abspath("__file__"))

# Trova il percorso assoluto della cartella "lib"
lib_dir = os.path.join(current_dir, "..", "lib")

# Aggiungi il percorso di "lib" al PYTHONPATH
sys.path.append(lib_dir)

from Utility import *

features = [
    'GoalsAvgSum', 'UltimoScontroDiretto', 'GoalRatioDifference', 'feat2', 'GoalCumulativeSum'
]

features2 = ['feat1', 'EwmaGoalsSum']

## DATA

In [667]:
data = getAltriData().copy()
# data = pd.read_excel('../data/mergedSerieA.xlsx')
data.tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BFECAHA,Referee,feat1,feat2,GoalsAvgSum,RecentFormRatio,GoalCumulativeSum,GoalCumulativeSumPrev,FormRatio,isOver
16559,SP2,22/09/2024,15:15,Cartagena,Cadiz,1,2,A,0.0,1.0,...,1.94,NaN,0.500000,0.625000,NaN,NaN,10,8.0,NaN,1
16560,SP2,22/09/2024,15:15,Castellon,Santander,0,1,A,0.0,0.0,...,1.94,NaN,1.333333,1.545455,NaN,NaN,17,10.0,NaN,0
16561,SP2,22/09/2024,17:30,Elche,Mirandes,1,0,H,0.0,0.0,...,1.98,NaN,5.357143,1.000000,NaN,NaN,9,17.0,NaN,0
16562,SP2,22/09/2024,20:00,Almeria,Eibar,2,2,D,0.0,1.0,...,2.05,NaN,2.666667,1.000000,NaN,NaN,13,9.0,NaN,1
16563,SP2,23/09/2024,19:30,Huesca,Cordoba,4,1,H,3.0,1.0,...,1.88,NaN,0.555556,1.100000,NaN,NaN,11,13.0,NaN,1


## Feature Engineering

In [668]:
data = data.dropna(subset=features)
# data = data.dropna()
print(len(data))
data['isOver'] = np.where(data['MatchGoal'] > 2.5, 1, 0)
print('Over:', round(100*data[data['isOver']==1]['isOver'].count() / len(data['isOver'])), '%')
x_train, x_test, y_train, y_test = train_test_split(
    data[features], data['isOver'], test_size=0.3, random_state=42, shuffle=True)

x_train = x_train.sort_index()
x_test = x_test.sort_index()
y_train = y_train.sort_index()
y_test = y_test.sort_index()


print(f'X_train: {x_train.shape} \nX_test: {x_test.shape} \ny_train: {y_train.shape} \ny_test: {y_test.shape}')

6983
Over: 46 %
X_train: (4888, 5) 
X_test: (2095, 5) 
y_train: (4888,) 
y_test: (2095,)


## Models

### Decision Tree model

In [669]:
from sklearn.tree import DecisionTreeClassifier



# Crea e addestra il modello di albero di decisione per la regressione
decTreeModel = DecisionTreeClassifier(random_state=42, min_samples_split=80
                                 # , class_weight={1: 1.2, 0: 1}
                                 # , class_weight='balanced'
                                 , max_depth=3  # , ccp_alpha= 0.00001
                                 # , max_features= 3
                                 ).fit(x_train, y_train)

# decTreeModel = RandomForestClassifier(random_state=42, n_estimators=100, 
#                                 min_samples_split=80, max_depth=3).fit(x_train_under, y_train_under) 

# dumpModel("Step1.joblib", decTreeModel)
# decTreeModel = loadModel("Step1.joblib")

### LogisticRegression

In [670]:
logRegModel = trainLogRegModel(x_train, y_train)

### Random Forest

In [671]:
# Crea e addestra il modello di Random Forest


from sklearn.model_selection import GridSearchCV

rfModel = RandomForestClassifier(random_state=42, n_estimators=200, 
                                min_samples_split=20, max_depth=3,
                                class_weight='balanced_subsample'
                                ).fit(x_train, y_train) 

# param_grid = {
#     'n_estimators': [200],
#     'min_samples_split': [50, 100],
#     'max_depth': [3, 5],
# }

# grid_search = GridSearchCV(estimator=rfModel, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(x_train, y_train)

# print("Migliori iperparametri:", grid_search.best_params_)
# print("Migliore accuratezza:", grid_search.best_score_)

# best_rfModel = grid_search.best_estimator_

# rfModel = best_rfModel


# dumpModel("Step1RF.joblib", rfModel)
rfModel = loadModel('Step1RF.joblib')

In [672]:
from sklearn.multiclass import OneVsRestClassifier

# Crea un classificatore One-vs-Rest usando Logistic Regression come classificatore base
ovr_classifier = OneVsRestClassifier(LogisticRegression())

# Addestra il classificatore
ovr_classifier.fit(x_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression())

## Utility Func for Gain

In [673]:
quotaMin = 1.50
def calculate_gain_O25(row, prediction):
    
    if (row[prediction] == 1):
        if row['B365>2.5'] > quotaMin :
            if row['MatchGoal'] > 2.5:
                return row['B365>2.5']-1
            else:
                return -1
        else:
            return 0
    elif (row[prediction] == 0): 
        if row['B365<2.5'] > quotaMin :
            if row['MatchGoal'] < 2.5:
                return row['B365<2.5']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0

def calculate_gain_Over25(row, prediction):
    # print('row', row['B365>2.5'], pd.isna(row['B365>2.5']))
    if (row[prediction] == 1):
        # return 0
        if row['B365>2.5'] > quotaMin :
            if row['MatchGoal'] > 2.5:
                return row['B365>2.5']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0
    
def calculate_gain_Under25(row, prediction):
    if (row[prediction] == 0): 
        # return 0
        if row['B365<2.5'] > quotaMin :
            if row['MatchGoal'] < 2.5:
                return row['B365<2.5']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0

## Metrics

In [674]:
from itertools import count
from math import nan
from sklearn.metrics import accuracy_score

df_train = data[data.index.isin(x_train.index)].copy()
print(len(df_train))

df_train['LogRegPred'] = logRegModel.predict(x_train)
df_train['DecTreePred'] = decTreeModel.predict(x_train)
df_train['RFPred'] = rfModel.predict(x_train)


df_test = data[data.index.isin(x_test.index)].copy()
print(len(df_test))

df_test['LogRegPred'] = logRegModel.predict(x_test)
df_test['DecTreePred'] = decTreeModel.predict(x_test)
df_test['RFPred'] = rfModel.predict(x_test)
df_test['1vsAllPred'] = ovr_classifier.predict(x_test)
df_test['FullPred'] = np.where(df_test['RFPred']+df_test['LogRegPred'] ==2, 1, 0)

print(
    f"Test LogReg Accuracy score: {round(100*accuracy_score(y_test, df_test['LogRegPred']),2)}%")
print(
    f"Test DecTree Accuracy score: {round(100*accuracy_score(y_test, df_test['DecTreePred']),2)}%")
print(
    f"Train Random Forest Accuracy score: {round(100*accuracy_score(y_train, df_train['RFPred']),2)}%")
print(
    f"Test Random Forest Accuracy score: {round(100*accuracy_score(y_test, df_test['RFPred']),2)}%")
print(
    f"Test OneVsAll Accuracy score: {round(100*accuracy_score(y_test, df_test['1vsAllPred']),2)}%")
print(
    f"Test FullPred Accuracy score: {round(100*accuracy_score(y_test, df_test['FullPred']),2)}%")

models = ['LogReg', 'RF', 'DecTree', '1vsAll']
rows = []
classAccuracyDf = pd.DataFrame(
    columns=['Modello', 'Accuratezza_Under', 'Accuratezza_Over'])
for model in models:
    row = {
        "Modello": model,
        "Accuratezza_Over": class_accuracy(y_test, df_test[model+'Pred'])['Over'],
        "Accuratezza_Under": class_accuracy(y_test, df_test[model+'Pred'])['Under']
    }
    rows.append(row)

classAccuracyDf = pd.DataFrame(rows)
classAccuracyDf.set_index('Modello', inplace=True)

print('')
print(classAccuracyDf)


prediction = 'DecTreePred'
prediction = 'RFPred'

# df_test['B365<2.5'] = 1.9
df_test['B365<2.5'] = np.where(pd.isna(df_test['B365<2.5']), 1.9, df_test['B365<2.5'])
df_test['B365>2.5'] = np.where(pd.isna(df_test['B365>2.5']), 1.9, df_test['B365>2.5'])


df_test['GainOver'] = df_test.apply(calculate_gain_Over25, axis=1, args=(prediction,) )
df_test['GainUnder'] = df_test.apply(calculate_gain_Under25, axis=1, args=(prediction,) )
# df_test['Gain'] = df_test['GainOver'].loc(-1)+ df_test_under['GainUnder'].loc(-1) 
# df_test['Equity'] = df_test['Gain'].cumsum()
df_test['EquityOver'] = df_test['GainOver'].cumsum()
df_test['EquityUnder'] = df_test['GainUnder'].cumsum()

# print(df_test['Equity'].tail(1))
print(df_test['EquityOver'].tail(1))
print(df_test['EquityUnder'].tail(1))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_test['isOver'], df_test[prediction])
print(cm)

train_to_export = df_train[df_train[prediction] ==1].copy()
print(len(train_to_export))
# train_to_export.to_excel("../Dataframe/Train Over Step1.xlsx")
test_to_export = df_test[df_test[prediction] ==1].copy()
print(len(test_to_export))
# test_to_export.to_excel("../Dataframe/Test Over Step1.xlsx")

# exportExcelWithTimeStamp(df_test, "../Dataframe/", "df.xlsx")

4888
2095
Test LogReg Accuracy score: 54.27%
Test DecTree Accuracy score: 53.17%
Train Random Forest Accuracy score: 52.56%
Test Random Forest Accuracy score: 52.46%
Test OneVsAll Accuracy score: 54.27%
Test FullPred Accuracy score: 54.27%

         Accuratezza_Over  Accuratezza_Under
Modello                                     
LogReg           0.578035           0.539542
RF               0.493567           0.545968
DecTree          0.510000           0.534037
1vsAll           0.578035           0.539542
16485   -62.69
Name: EquityOver, dtype: float64
16485   -6.32
Name: EquityUnder, dtype: float64
[[677 433]
 [563 422]]
2001
855


## Second Round

In [675]:
dataTrain = train_to_export#getTrainOver1Data().copy()
dataTest = test_to_export#getTestOver1Data().copy()
dataTrain = dataTrain.dropna(subset=features2)
dataTest = dataTest.dropna(subset=features2)
# data2 = df_merged[df_merged[prediction] ==1].copy()


# data = data.dropna()

x_train = dataTrain[features2]
y_train = dataTrain['isOver']
x_test = dataTest[features2]
y_test = dataTest['isOver']

x_train = x_train.sort_index()
x_test = x_test.sort_index()
y_train = y_train.sort_index()
y_test = y_test.sort_index()

print(f'X_train: {x_train.shape} \nX_test: {x_test.shape} \ny_train: {y_train.shape} \ny_test: {y_test.shape}')

X_train: (2001, 2) 
X_test: (855, 2) 
y_train: (2001,) 
y_test: (855,)


In [676]:
# Crea e addestra il modello di albero di decisione per la regressione
decTreeModel = DecisionTreeClassifier(random_state=42, min_samples_split=80
                                 # , class_weight={1: 1.2, 0: 1}
                                 # , class_weight='balanced'
                                 , max_depth=3  # , ccp_alpha= 0.00001
                                 # , max_features= 3
                                 ).fit(x_train, y_train)

# decTreeModel = RandomForestClassifier(random_state=42, n_estimators=100, 
#                                 min_samples_split=80, max_depth=3).fit(x_train_under, y_train_under) 

# dumpModel("Under.joblib", decTreeModel)
# decTreeModel = loadModel("Under.joblib")
logRegModel = trainLogRegModel(x_train, y_train)
# dumpModel("Step2.joblib", decTreeModel)
# logRegModel = loadModel("Step2.joblib")

# Crea e addestra il modello di Random Forest
rfModel2 = RandomForestClassifier(random_state=42, n_estimators=100, 
                                min_samples_split=80, max_depth=3).fit(x_train, y_train) 

# dumpModel("Over.joblib", rfModel2)
# rfModel2 = loadModel('Over.joblib')

In [677]:
from itertools import count
from sklearn.metrics import accuracy_score
df_test = dataTest[dataTest.index.isin(x_test.index)].copy()
print(len(df_test))

df_test['LogRegPred'] = logRegModel.predict(x_test)
df_test['DecTreePred'] = decTreeModel.predict(x_test)
df_test['RFPred'] = rfModel2.predict(x_test)



print(
    f"Test LogReg Accuracy score: {round(100*accuracy_score(y_test, df_test['LogRegPred']),2)}%")
print(
    f"Test DecTree Accuracy score: {round(100*accuracy_score(y_test, df_test['DecTreePred']),2)}%")
print(
    f"Test Random Forest Accuracy score: {round(100*accuracy_score(y_test, df_test['RFPred']),2)}%")

models = ['LogReg', 'RF', 'DecTree']
rows = []
classAccuracyDf = pd.DataFrame(
    columns=['Modello', 'Accuratezza_Under', 'Accuratezza_Over'])
for model in models:
    row = {
        "Modello": model,
        "Accuratezza_Over": class_accuracy(y_test, df_test[model+'Pred'])['Over'],
        "Accuratezza_Under": class_accuracy(y_test, df_test[model+'Pred'])['Under']
    }
    rows.append(row)

classAccuracyDf = pd.DataFrame(rows)
classAccuracyDf.set_index('Modello', inplace=True)

print('')
print(classAccuracyDf)


prediction = 'DecTreePred'
# prediction = 'RFPred'
# prediction = 'LogRegPred'

df_test['B365<2.5'] = 1.8
df_test['B365>2.5'] = 1.8


df_test['GainOver'] = df_test.apply(calculate_gain_Over25, axis=1, args=(prediction,) )
df_test['GainUnder'] = df_test.apply(calculate_gain_Under25, axis=1, args=(prediction,) )
# df_test['Gain'] = df_test['GainOver'].loc(-1)+ df_test_under['GainUnder'].loc(-1) 
# df_test['Equity'] = df_test['Gain'].cumsum()
df_test['EquityOver'] = df_test['GainOver'].cumsum()
df_test['EquityUnder'] = df_test['GainUnder'].cumsum()

# print(df_test['Equity'].tail(1))
print(df_test['EquityOver'].tail(1))
print(df_test['EquityUnder'].tail(1))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_test['isOver'], df_test[prediction])
print(cm)
# exportExcelWithTimeStamp(df_test_under, "../Dataframe/", "df.xlsx")



855
Test LogReg Accuracy score: 53.1%
Test DecTree Accuracy score: 50.76%
Test Random Forest Accuracy score: 52.87%

         Accuratezza_Over  Accuratezza_Under
Modello                                     
LogReg           0.567742           0.522857
RF               0.587156           0.520107
DecTree          0.506173           0.507752
16481   -7.2
Name: EquityOver, dtype: float64
16481   -66.6
Name: EquityUnder, dtype: float64
[[393  40]
 [381  41]]
